<a href="https://colab.research.google.com/github/ofcucuh/webscrap-Mamikos.com/blob/main/Web_Scrapping_Mamikos_com.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium

In [2]:
import time

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import requests
import json
import pandas as pd

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--disable-extensions")
options.add_argument("--disable-infobars")
options.add_argument("--disable-notifications")
options.add_argument("--disable-popup-blocking")

In [3]:
from bs4 import BeautifulSoup

def get_detail(response, url, kost_data=None):
    headers = {
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36",
      "Accept-Language": "en-US,en;q=0.9",
      "Connection": "keep-alive"
    }

    soup = BeautifulSoup(response.text, "html.parser")
    script_tags = soup.find_all("script", string=re.compile(r"var detail ="))
    jsonDataDetail = None

    #ambil js yang ada di tag <script> setelah akses ke url
    for script in script_tags:
        script_content = script.string
        match = re.search(r"var detail = (\{.*\});", script_content)
        if match:
            detail_json_str = match.group(1)
            detail_data = json.loads(detail_json_str)
            jsonDataDetail = detail_data
            break

    if not jsonDataDetail:
        print("Tidak dapat menemukan detail data.")
        print(url)
        kost_data["status_cek"] = "FAIL LAGI"
        return kost_data

    id_kost = jsonDataDetail.get("_id")
    if not id_kost:
        kost_data["status_cek"] = "FAIL LAGI"
        print("ID kost tidak ditemukan.")
        return kost_data

    # akses url review
    urlReview = f"https://mamikos.com/garuda/stories/{id_kost}/reviews"
    responseReview = requests.get(urlReview, headers=headers)
    reviewData = None

    if responseReview.status_code == 200:
        try:
            data = responseReview.json()
            reviewData = data
        except json.JSONDecodeError:
            print("Respons dari server bukan JSON:")
            print(responseReview.text)
    else:
        print(f"Request gagal dengan status code: {responseReview.status_code}")

    #proses datanya
    fasilitas = {}

    fasilitas["kamar"] = jsonDataDetail.get("fac_room", "Tidak ada data")
    fasilitas["umum"] = jsonDataDetail.get("fac_share", "Tidak ada data")
    fasilitas["kamar_mandi"] = jsonDataDetail.get("fac_bath", "Tidak ada data")
    fasilitas["parkir"] = jsonDataDetail.get("fac_park", "Tidak ada data")
    deskripsi = jsonDataDetail.get("description", "Tidak ada deskripsi")
    deskripsi_add = jsonDataDetail.get("remarks", "Tidak ada tambahan")

    harga = {
        "harian": jsonDataDetail.get("price_daily", "Tidak ada harga"),
        "mingguan": jsonDataDetail.get("price_weekly", "Tidak ada harga"),
        "bulanan": jsonDataDetail.get("price_monthly", "Tidak ada harga"),
        "3_bulan": jsonDataDetail.get("price_quarterly", "Tidak ada harga"),
        "6_bulan": jsonDataDetail.get("price_semiannualy", "Tidak ada harga"),
        "tahun": jsonDataDetail.get("price_yearly", "Tidak ada harga"),
    }

    kost_data["review"] = reviewData["all_rating"]
    kost_data["fasilitas"] = fasilitas
    kost_data["deskripsi"] = deskripsi
    kost_data["deskripsi_additional"] = deskripsi_add
    kost_data["harga_kost"] = harga
    kost_data["ukuran"] = jsonDataDetail.get("size", "Tidak ada ukuran")
    kost_data["url"] = url
    return kost_data


In [ ]:
# test ambil data detail
# get_detail("https://mamikos.com/room/kost-kota-bandung-kost-campur-murah-kost-apik-mekar-ii-padasuka-tipe-a-cibeunying-kidul-bandung-2")

In [4]:
# buat variabel global biar inget
kost_data = []
new_kost_data = []

In [5]:
def getPageData(url,area,kota):
  driver = webdriver.Chrome(options=options)
  all_kost_data = []
  try:
    #sorting & filtering mamikos
      driver.get(url+"?gender=0,1,2&rent=2&sort=price,asc&price=10000-10000000&singgahsini=0")
      print("Menunggu elemen 'room-list__card' muncul...")
      WebDriverWait(driver, 5).until(
          EC.presence_of_element_located((By.CLASS_NAME, "room-list__card"))
      )
      WebDriverWait(driver, 0.5).until(
          EC.presence_of_element_located((By.CLASS_NAME, "pagination"))
      )

      slider_div = driver.find_element(By.CLASS_NAME, "custom-range-container")
      slider = slider_div.find_element(By.TAG_NAME, "input")
      actions = ActionChains(driver)

      #ubah ke 1KM
      actions.click_and_hold(slider).move_by_offset(-350, 0).release().perform()

      WebDriverWait(driver, 5).until(
          EC.presence_of_element_located((By.CLASS_NAME, "room-list__card"))
      )

      #scroll sampe muncul popup
      while True:
          driver.execute_script("window.scrollBy(0,100);")
          try:
              popup = driver.find_element(By.CLASS_NAME, "popper-ftue")
              if popup.is_displayed():
                  print("Popup appeared!")
                  popupBtn = popup.find_element(By.CLASS_NAME, "popper-ftue__content-button")
                  popupBtn.click()
                  break
          except:
              pass
          time.sleep(0.3)

      #cek elmen pagination
      next_element = driver.find_element(By.CLASS_NAME, "pagination")
      next = next_element.find_elements(By.TAG_NAME, "li")
      last_element = next[-1]
      i=0

      #proses data
      while True:
          if "disabled" in last_element.get_attribute("class"):
              break
          WebDriverWait(driver, 5).until(
              EC.presence_of_element_located((By.CLASS_NAME, "room-list-container"))
          )
          kost_elements = driver.find_elements(By.CLASS_NAME, "room-list__card")
          for kost in kost_elements:
                jenis_kost = "tanpa-jenis"
                #ambil jenis berdasarkan iconnya
                try:
                    WebDriverWait(driver, 3).until(
                        EC.presence_of_element_located((By.CLASS_NAME, "rc-photo__controlled-room"))
                    )
                    jenis_elem = kost.find_element(By.CLASS_NAME, "rc-photo__controlled-room__brand-icon")
                    img_src = jenis_elem.find_element(By.TAG_NAME, "img").get_attribute("data-src")
                    if "singgahsini" in img_src.lower():
                        jenis_kost = "singgahsini"
                    elif "apik" in img_src.lower():
                        jenis_kost = "apik"
                except:
                    jenis_kost = "tanpa-jenis"

                #ambil tipe berdasarkan labelnya
                tipe_kost = "campur"
                tipe_elem = kost.find_elements(By.CLASS_NAME, "rc-overview__label")
                if tipe_elem:
                    tipe_text = tipe_elem[0].text.lower()
                    if "putra" in tipe_text:
                        tipe_kost = "putra"
                    elif "putri" in tipe_text:
                        tipe_kost = "putri"

                #ambil harga, kemudian beri label jika > 1000000/bulan = eksklusif
                harga_kost = "murah"
                harga_elem = kost.find_element(By.CLASS_NAME, "rc-price__text").text
                harga_numerik = int(re.sub(r'\D', '', harga_elem))
                if harga_numerik >= 999999:
                    harga_kost = "eksklusif"

                #ambil nama
                nama_kost_elem = kost.find_element(By.CLASS_NAME, "rc-info__name").text.lower()
                nama_kost = re.sub(r'\.', '', nama_kost_elem)
                nama_kost = re.sub(r'[^a-z0-9]+', '-', nama_kost).strip('-')
                nama_kost = nama_kost.replace("kost-", "")

                #ambil lokasi yang tercantum
                lokasi_kost_elem = kost.find_element(By.CLASS_NAME, "rc-info__location").text.lower()
                lokasi_kost = re.sub(r'[^a-z0-9]+', '-', lokasi_kost_elem).strip('-')
                lokasi_kost = lokasi_kost.replace("kecamatan-", "").replace("kelurahan-", "")

                #generate urlnya berdasarkan pattern  kost-kota-{area}-{tipe_kost}-{harga_kost}-kost-{jenis_kost}-{nama_kost}-{lokasi_kost}-{area}
                #some case gapake kota jadi           kost-{area}-lalala-{area}
                #another case pake kabupaten jadi     kost-kabupaten-{area}-lalala-{area}
                #sesuaikan urlnya aja

                kost_url = f"https://mamikos.com/room/kost-{kota}-{area}-kost-{tipe_kost}-{harga_kost}-kost-{jenis_kost}-{nama_kost}-{lokasi_kost}-{area}"
                kost_url = kost_url.replace("-tanpa-jenis","")
                kost = {
                    "url":kost_url,
                    "nama":nama_kost,
                    "lokasi_kost":lokasi_kost,
                    "harga_kost": {},
                    "status_cek":None, #sukses/fail
                    "deskripsi":None, #cerita pemilik
                    "deskripsi_additional":None, #Catatan tambahan seputar fasilitas
                    "fasilitas": {}, #fac lalala
                    "review":{} #kebersihan, kenyamanan lalala
                }
                all_kost_data.append(kost)

          #click next button untuk halaman selanjutnya
          link = last_element.find_element(By.TAG_NAME, "a")
          link.click()

          WebDriverWait(driver, 5).until(
              EC.presence_of_element_located((By.CLASS_NAME, "room-list__card"))
          )
          WebDriverWait(driver, 0.5).until(
              EC.presence_of_element_located((By.CLASS_NAME, "pagination"))
          )
          next_element = driver.find_element(By.CLASS_NAME, "pagination")
          next = next_element.find_elements(By.TAG_NAME, "li")
          last_element = next[-1]
  except Exception as e:
      print(f"Terjadi kesalahan saat memuat halaman utama: {e}")

  finally:
      driver.quit()

  #simpen dulu biar bisa lanjut nanti
  dfAllData = pd.DataFrame(all_kost_data)
  dfAllData.to_excel("/content/drive/MyDrive/BatchScrapt/"+url.replace("https://mamikos.com/kost/","")+".xlsx", index=False)

In [ ]:
#surabaya
urlKost = [
    "https://mamikos.com/kost/kost-dekat-unair-surabaya-murah",
    "https://mamikos.com/kost/kost-dekat-its-surabaya-murah",
    "https://mamikos.com/kost/kost-dekat-unesa-surabaya-murah",
    "https://mamikos.com/kost/kost-dekat-upn-surabaya-murah",
    "https://mamikos.com/kost/kost-dekat-ubaya-universitas-surabaya-murah",
    "https://mamikos.com/kost/kost-dekat-uin-surabaya-murah",
    "https://mamikos.com/kost/kost-dekat-uwm-surabaya-murah",
    "https://mamikos.com/kost/kost-dekat-uk-petra-surabaya-murah",]
for url in urlKost:
  getPageData(url,"surabaya","kota")

Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!


In [ ]:
#jogja
urlKost = [
    "https://mamikos.com/kost/kost-dekat-ugm-murah",
    "https://mamikos.com/kost/kost-dekat-uny-jogja-murah",
    "https://mamikos.com/kost/kost-dekat-umy-murah",
    "https://mamikos.com/kost/kost-dekat-uin-jogja-murah",
    "https://mamikos.com/kost/kost-dekat-upn-jogja-murah",
    "https://mamikos.com/kost/kost-dekat-usd-jogja-murah",
    "https://mamikos.com/kost/kost-dekat-uad-jogja-murah",
    "https://mamikos.com/kost/kost-dekat-uty-jogja-murah",
    "https://mamikos.com/kost/kost-dekat-atmajaya-jogja-murah",
    "https://mamikos.com/kost/kost-dekat-stie-ykpn-jogja-murah",]
for url in urlKost:
  getPageData(url,"yogyakarta","kota")

Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!


In [ ]:
#jakarta
urlKost = [
    "https://mamikos.com/kost/kost-dekat-stan-jakarta-murah",
    "https://mamikos.com/kost/kost-dekat-uin-jakarta-murah",
    "https://mamikos.com/kost/kost-dekat-unj-jakarta-murah",
    "https://mamikos.com/kost/kost-dekat-binus-kemanggisan-murah",
    "https://mamikos.com/kost/kost-dekat-ui-depok-murah",
    "https://mamikos.com/kost/kost-dekat-binus-alam-sutra-murah",
    "https://mamikos.com/kost/kost-dekat-atma-jaya-jakarta-murah",
    "https://mamikos.com/kost/kost-dekat-trisakti-murah",
    "https://mamikos.com/kost/kost-dekat-untar-murah",
    "https://mamikos.com/kost/kost-dekat-uph-jakarta-murah",
    "https://mamikos.com/kost/kost-dekat-gunadarma-kelapa-dua-murah"]
for url in urlKost:
  getPageData(url,"jakarta","kota")

In [ ]:
#bandung
urlKost = [
    "https://mamikos.com/kost/kost-dekat-itb-murah",
    "https://mamikos.com/kost/kost-dekat-upi-bandung-murah",
    "https://mamikos.com/kost/kost-dekat-telkom-bandung-murah",
    "https://mamikos.com/kost/kost-dekat-uin-bandung-murah",
    "https://mamikos.com/kost/kost-dekat-unpar-bandung-murah",
    "https://mamikos.com/kost/kost-dekat-maranatha-bandung-murah",
    "https://mamikos.com/kost/kost-dekat-unpad-jatinangor-murah",
    "https://mamikos.com/kost/kost-dekat-unpad-dipatiukur-murah",
    "https://mamikos.com/kost/kost-dekat-unikom-bandung-murah",
    "https://mamikos.com/kost/kost-dekat-unisba-bandung-murah",
    "https://mamikos.com/kost/kost-dekat-unjani-bandung-murah",
    "https://mamikos.com/kost/kost-dekat-itenas-bandung-murah",
    "https://mamikos.com/kost/kost-dekat-widyatama-bandung-murah",]
for url in urlKost:
  getPageData(url,"bandung","kota")

In [ ]:
kabupaten_badung = [
    "https://mamikos.com/kost/kost-jimbaran-badung-bali-murah",
    "https://mamikos.com/kost/kost-kerobokan-badung-bali-murah",
    "https://mamikos.com/kost/kost-seminyak-kuta-badung-bali-murah",
    "https://mamikos.com/kost/kost-canggu-badung-bali-murah",
    "https://mamikos.com/kost/kost-kuta-badung-bali-murah"
]
for url in kabupaten_badung:
  getPageData(url,"badung","kabupaten")

kota_denpasar = [
    "https://mamikos.com/kost/kost-renon-denpasar-bali-murah",
    "https://mamikos.com/kost/kost-jalan-glogor-carik-bali-murah",
    "https://mamikos.com/kost/kost-panjer-denpasar-bali-murah",
    "https://mamikos.com/kost/kost-pantai-sanur-bali-murah",
    "https://mamikos.com/kost/kost-dekat-bandara-ngurah-rai-dps-bali-murah"
]
for url in kota_denpasar:
  getPageData(url,"denpasar","kota")

Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!


In [7]:
kota_malang = [
    "https://mamikos.com/kost/kost-suhat-malang-murah",
    "https://mamikos.com/kost/kost-batu-malang-murah",
    "https://mamikos.com/kost/kost-blimbing-malang-murah",
    "https://mamikos.com/kost/kost-sigura-gura-malang-murah",
    "https://mamikos.com/kost/kost-lowokwaru-malang-murah",
    "https://mamikos.com/kost/kost-klojen-malang-murah",
    "https://mamikos.com/kost/kost-singosari-malang-murah",
    "https://mamikos.com/kost/kost-sukun-malang-murah",
    "https://mamikos.com/kost/kost-sumbersari-malang-murah",
    "https://mamikos.com/kost/kost-dinoyo-malang-murah",
    "https://mamikos.com/kost/kost-dekat-stasiun-kepanjen-malang-murah"
]

kota_semarang = [
    "https://mamikos.com/kost/kost-tembalang-semarang-murah",
    "https://mamikos.com/kost/kost-ngaliyan-semarang-murah",
    "https://mamikos.com/kost/kost-banyumanik-semarang-murah",
    "https://mamikos.com/kost/kost-ungaran-barat-semarang-murah",
    "https://mamikos.com/kost/kost-pedurungan-semarang-murah",
    "https://mamikos.com/kost/kost-sampangan-semarang-murah",
    "https://mamikos.com/kost/kost-pleburan-semarang-selatan-murah",
    "https://mamikos.com/kost/kost-semarang-timur-murah",
    "https://mamikos.com/kost/kost-gajah-mungkur-semarang-murah",
    "https://mamikos.com/kost/kost-genuk-semarang-murah",
    "https://mamikos.com/kost/kost-gayamsari-semarang-murah",
    "https://mamikos.com/kost/kost-dekat-semarang-tawang-murah"
]

kota_surabaya = [
    "https://mamikos.com/kost/kost-rungkut-surabaya-murah",
    "https://mamikos.com/kost/kost-surabaya-barat-murah",
    "https://mamikos.com/kost/kost-ketintang-surabaya-murah",
    "https://mamikos.com/kost/kost-ngagel-surabaya-murah",
    "https://mamikos.com/kost/kost-gubeng-surabaya-murah",
    "https://mamikos.com/kost/kost-mulyorejo-surabaya-murah",
    "https://mamikos.com/kost/kost-wiyung-surabaya-murah",
    "https://mamikos.com/kost/kost-surabaya-timur-murah",
    "https://mamikos.com/kost/kost-dukuh-pakis-surabaya-murah",
    "https://mamikos.com/kost/kost-wonokromo-surabaya-murah",
    "https://mamikos.com/kost/kost-dekat-bandara-juanda-surabaya-murah",
    "https://mamikos.com/kost/kost-dekat-stasiun-gubeng-surabaya-murah"
]

for url in kota_malang:
  getPageData(url,"malang","kota")
for url in kota_semarang:
  getPageData(url,"semarang","kota")
for url in kota_surabaya:
  getPageData(url,"surabaya","kota")

Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul...
Popup appeared!
Menunggu elemen 'room-list__card' muncul

In [8]:
import pandas as pd
import os

def combine_excel_files(folder_path, output_file):
    all_dataframes = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".xlsx"):
            file_path = os.path.join(folder_path, filename)
            try:
                df = pd.read_excel(file_path)
                all_dataframes.append(df)
            except Exception as e:
                print(f"Error reading file {filename}: {e}")

    if not all_dataframes:
        print("No Excel files found in the specified folder.")
        return

    combined_df = pd.concat(all_dataframes, ignore_index=True)
    combined_df.to_excel(output_file, index=False)
    print(f"Combined Excel data saved to {output_file}")

folder_path = "/content/drive/MyDrive/BatchScrapt"
output_file = "/content/drive/MyDrive/BatchScrapt/combined_data.xlsx"
combine_excel_files(folder_path, output_file)

Combined Excel data saved to /content/drive/MyDrive/BatchScrapt/combined_data.xlsx


In [9]:
all_kost_data = pd.read_excel("/content/drive/MyDrive/BatchScrapt/combined_data.xlsx").to_dict(orient='records')
# pastikan dapet urlnya
for kost in all_kost_data:
  print(kost.get("url"))
  break
new_kost_data = []

https://mamikos.com/room/kost-kabupaten-badung-kost-putri-eksklusif-kost-pondok-ebira-standart-ac-kuta-utara-badung


In [ ]:
#ambil sampe 10 data buat coba coba
sample_data = all_kost_data[:10]

In [10]:
# proses satu satu, pastikan new_kost_data kosong, gunakan sample_data untuk testing
new_kost_data = []
fail_kost_data = []
# for kost in sample_data:
for kost in all_kost_data:
    try:
        headers = {
          "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36",
          # "Referer": "https://www.example.com",
          "Accept-Language": "en-US,en;q=0.9",
          "Connection": "keep-alive"
        }
        url = kost.get("url")
        response = requests.get(url, allow_redirects=True, headers=headers)
        time.sleep(2)
        final_url = response.url
        if(final_url == "https://mamikos.com"):
          kost["status_cek"] = "FAIL"
          # url2 = url.replace("kost-kota-","kost-")
          # response = requests.get(url2, allow_redirects=True, headers=headers)
          # time.sleep(1)
          # final_url = response.url
          # if(final_url == "https://mamikos.com"):
          #   kost["status_cek"] = "FAIL"
          #   fail_kost_data.append(kost)
          # else:
          #   kost["status_cek"] = "SUKSES"
          #   new_data = get_detail(response, url2, kost)
          #   new_kost_data.append(new_data)
        else:
          kost["status_cek"] = "SUKSES"
          new_data = get_detail(response, url, kost)
          new_kost_data.append(new_data)

    except Exception as e:
        print(f"[ERROR] Gagal mengakses URL: Error: {e}")

#yang gagal disimpen buat diubah urlnya manual

Tidak dapat menemukan detail data.
https://mamikos.com/room/kost-kabupaten-badung-kost-campur-eksklusif-kost-the-aulia-stay-kuta-selatan-badung
Tidak dapat menemukan detail data.
https://mamikos.com/room/kost-kabupaten-badung-kost-campur-eksklusif-kost-graha-pertiwi-tipe-a-kuta-utara-badung
Tidak dapat menemukan detail data.
https://mamikos.com/room/kost-kota-denpasar-kost-campur-eksklusif-kost-oemah-fazilla-denpasar-selatan-denpasar
Tidak dapat menemukan detail data.
https://mamikos.com/room/kost-kota-denpasar-kost-campur-eksklusif-kost-pondok-48-denpasar-selatan-denpasar
Tidak dapat menemukan detail data.
https://mamikos.com/room/kost-kota-denpasar-kost-campur-murah-kost-cahaya-jiwa-tipe-a-denpasar-barat-denpasar
Tidak dapat menemukan detail data.
https://mamikos.com/room/kost-kota-malang-kost-putra-murah-kost-pak-ateng-lowokwaru-malang
Tidak dapat menemukan detail data.
https://mamikos.com/room/kost-kota-malang-kost-putra-murah-kost-kerto-asri-tipe-a-lowokwaru-malang
Tidak dapat men

In [11]:
#beautify
def process_kost_data(kost_data):
    """Processes the kost data and returns a structured DataFrame."""
    processed_data = []

    for kost in kost_data:
        if kost is None:
            continue  # Skip if kost data is None
        if kost.get("status_cek") != "SUKSES":
            continue  # Skip if status_cek is not "SUKSES"

        # Initialize dictionary for processed kost
        processed_kost = {
            'url': kost.get('url', ''),
            'nama': kost.get('nama', ''),
            'alamat': kost.get('lokasi_kost', None),  # Use None for missing data
            'lokasi_kost': kost.get('lokasi_kost', ''),
            'status_cek': kost.get('status_cek', ''),
            'deskripsi': kost.get('deskripsi', None),  # Handle missing description
            'deskripsi_additional': kost.get('deskripsi_additional', None),
            'ukuran': kost.get('ukuran', None),  # Handle missing ukuran
        }

        # Process harga_kost (Ensure it is always a dictionary)
        harga = kost.get('harga_kost', '{}')  # Default to '{}' if missing
        if isinstance(harga, str):  # If it's a string, try to convert it
            try:
                harga = json.loads(harga)  # Convert string to dictionary if it's a valid JSON string
            except json.JSONDecodeError:
                harga = {}  # If it can't be decoded, use an empty dictionary

        # Process the harga_kost dictionary
        for key, value in harga.items():
            processed_kost[f'harga_{key}'] = value

        # Process fasilitas (Handle missing or empty categories)
        fasilitas = kost.get('fasilitas', '{}')  # Default to '{}' if missing
        if isinstance(fasilitas, str):  # If it's a string, try to convert it
            try:
                fasilitas = json.loads(fasilitas)  # Convert string to dictionary if it's a valid JSON string
            except json.JSONDecodeError:
                fasilitas = {}  # If it can't be decoded, use an empty dictionary

        # Process the fasilitas dictionary
        for category, items in fasilitas.items():
            processed_kost[f'fasilitas_{category}'] = ', '.join(items) if isinstance(items, list) else ''

        # Process review (Handle missing or empty reviews)
        review = kost.get('review', '{}')  # Default to '{}' if missing
        if isinstance(review, str):  # If it's a string, try to convert it
            try:
                review = json.loads(review)  # Convert string to dictionary if it's a valid JSON string
            except json.JSONDecodeError:
                review = {}  # If it can't be decoded, use an empty dictionary

        # Process the review dictionary
        for key, value in review.items():
            if key:  # Skip empty key
                processed_kost[f'review_{key}'] = value

        # Append the processed kost to the list
        processed_data.append(processed_kost)

    # Convert processed data to DataFrame
    return pd.DataFrame(processed_data)

In [ ]:
# print(new_kost_data)

In [12]:
df = process_kost_data(new_kost_data)
df.to_excel("/content/drive/MyDrive/BatchScraptBanyak.xlsx", index=False)